In [ ]:
!pip3 install numpy opencv-python matplotlib scikit-learn

  Using cached numpy-2.1.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached matplotlib-3.9.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (13 kB)
  Using cached contourpy-1.3.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.0-cp312-cp312-macosx_10_13_universal2.whl.metadata (164 kB)
  Using cached kiwisolver-1.4.7-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached pillow-11.0.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached scipy-1.14.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.meta

In [11]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA


In [ ]:
# Load the original image in RGB format
img_path = '../assets/dog.jpg'
org_img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
org_img.shape

(3552, 5321, 3)

In [ ]:
# in the following we are doing the following steps to achieve compression:
#==1==converting the entire image to grayscale,
#==2==applying PCA for dimensionality reduction,
#==2==and then mapping it back to grayscale.

# Convert the image to grayscale
gray_img = cv2.cvtColor(org_img, cv2.COLOR_RGB2GRAY)

# Set the number of principal components for PCA
num_pc = 100  # Adjust based on testing to balance quality and compression

# Apply PCA to the grayscale image
pca = PCA(n_components=num_pc)
gray_reduced = pca.fit_transform(gray_img)
gray_reconstructed = pca.inverse_transform(gray_reduced)

# Clip values to ensure they are within the valid range
gray_reconstructed = np.clip(gray_reconstructed, 0, 255).astype(np.uint8)

# Convert the grayscale image back to RGB format for saving
comp_img_rgb = cv2.cvtColor(gray_reconstructed, cv2.COLOR_GRAY2RGB)

# Define the save path
save_path = '../processed_images/dog_compressed_grayscale.jpg'

# Save the compressed image with moderate JPEG quality
cv2.imwrite(save_path, comp_img_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])

# Display sizes to compare
original_size = os.path.getsize(img_path) / 1024  # Original size in KB
compressed_size = os.path.getsize(save_path) / 1024  # Compressed size in KB
print(f"Original Size: {original_size:.2f} KB")
print(f"Compressed Size: {compressed_size:.2f} KB")
print(f"Size is reduced by: {(100-(compressed_size*100)/original_size):.2f}%")


Original Size: 1183.70 KB
Compressed Size: 661.73 KB
Size is reduced by: 44.10%
Compressed grayscale image saved at: ../processed_images/dog_compressed_grayscale.jpg


In [18]:
#This method will result in a compressed color image without
# any significant distortions (like the code you had where the image was "aggressively compressed").

# Split the image into R, G, B channels
r, g, b = cv2.split(org_img)

# Set the number of principal components for PCA
num_pc = 100  # Adjust based on testing to balance quality and compression

# Apply PCA to each channel
pca_r = PCA(n_components=num_pc)
r_reduced = pca_r.fit_transform(r)
r_reconstructed = pca_r.inverse_transform(r_reduced)

pca_g = PCA(n_components=num_pc)
g_reduced = pca_g.fit_transform(g)
g_reconstructed = pca_g.inverse_transform(g_reduced)

pca_b = PCA(n_components=num_pc)
b_reduced = pca_b.fit_transform(b)
b_reconstructed = pca_b.inverse_transform(b_reduced)

# Clip values to ensure they are within the valid range [0, 255]
r_reconstructed = np.clip(r_reconstructed, 0, 255).astype(np.uint8)
g_reconstructed = np.clip(g_reconstructed, 0, 255).astype(np.uint8)
b_reconstructed = np.clip(b_reconstructed, 0, 255).astype(np.uint8)

# Merge the reconstructed channels back into an RGB image
comp_img_rgb = cv2.merge([r_reconstructed, g_reconstructed, b_reconstructed])

# Define the save path
save_path = '../processed_images/dog_compressed_colored.jpg'

# Save the compressed image with moderate JPEG quality
cv2.imwrite(save_path, cv2.cvtColor(comp_img_rgb, cv2.COLOR_RGB2BGR), [int(cv2.IMWRITE_JPEG_QUALITY), 60])

# Display sizes to compare
original_size = os.path.getsize(img_path) / 1024  # Original size in KB
compressed_size = os.path.getsize(save_path) / 1024  # Compressed size in KB
print(f"Original Size: {original_size:.2f} KB")
print(f"Compressed Size: {compressed_size:.2f} KB")
print(f"Size is reduced by: {(100-(compressed_size*100)/original_size):.2f}%")


Original Size: 1183.70 KB
Compressed Size: 669.74 KB
Size is reduced by: 43.42%
